In [26]:
import pandas as pd

#Merge 3 dataframes into 1                
df = pd.read_csv("./data/User_track_data.csv")
df1 = pd.read_csv("./data/User_track_data_2.csv")
m1 = pd.merge(df,df1,how="outer") # m1 = df+df1
df2 = pd.read_csv("./data/User_track_data_3.csv")
mega_df = pd.merge(m1,df2,how="outer") # mega_df = df2+m1

print(mega_df.tail(5))

    User_name                  User_ID   Artist_name                Track  \
295         J  jimenezzuletajhonjanner    Katy Perry     California Gurls   
296         J  jimenezzuletajhonjanner         Adele  Rolling in the Deep   
297         J  jimenezzuletajhonjanner   John Legend            All of Me   
298         J  jimenezzuletajhonjanner  Fools Garden           Lemon Tree   
299         J  jimenezzuletajhonjanner     Passenger             Let's Go   

     valence  
295    0.425  
296    0.507  
297    0.331  
298    0.643  
299    0.649  


In [55]:
# Antes del cambio del User_ID
df

,User_name,User_ID,Artist_name,Track,valence
0,Sebastian Racedo Val,12179172375,Nickelback,Animals,0.818
1,Sebastian Racedo Val,12179172375,Nickelback,Burn It to the Ground,0.600
2,Sebastian Racedo Val,12179172375,Panic! At The Disco,I Write Sins Not Tragedies,0.672
3,Sebastian Racedo Val,12179172375,Fall Out Boy,Thnks fr th Mmrs,0.588
4,Sebastian Racedo Val,12179172375,Hugo,99 Problems,0.592
...,...,...,...,...,...
95,J,jimenezzuletajhonjanner,El Cuarteto De Nos,Ya No Sé Que Hacer Conmigo,0.502
96,J,jimenezzuletajhonjanner,El Cuarteto De Nos,Mario Neta,0.418
97,J,jimenezzuletajhonjanner,El Cuarteto De Nos,Punta Cana,0.948
98,J,jimenezzuletajhonjanner,Residente,Pecador,0.465


In [48]:
def new_id(text: str) -> str:
    '''
    Devuelve un ID en su formato ASCII

    Args:
        text (str) : [Text that we'll transform into ASCII representation]
    '''
    carSplit = [letra for letra in text]
    ascii_rep = [ord(k) for k in carSplit]
    nuevoID = ''.join([str(i) for i  in ascii_rep])

    return nuevoID

In [56]:
for k in range(len(df['User_ID'])):
    df['User_ID'][k] = new_id(df['User_ID'][k])

# Despues del cambio del User_ID
df
    

/tmp/ipykernel_1658/1431246028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User_ID'][k] = new_id(df['User_ID'][k])


,User_name,User_ID,Artist_name,Track,valence
0,Sebastian Racedo Val,4950495557495550515553,Nickelback,Animals,0.818
1,Sebastian Racedo Val,4950495557495550515553,Nickelback,Burn It to the Ground,0.600
2,Sebastian Racedo Val,4950495557495550515553,Panic! At The Disco,I Write Sins Not Tragedies,0.672
3,Sebastian Racedo Val,4950495557495550515553,Fall Out Boy,Thnks fr th Mmrs,0.588
4,Sebastian Racedo Val,4950495557495550515553,Hugo,99 Problems,0.592
...,...,...,...,...,...
95,J,1061051091011101011221221171081011169710610411...,El Cuarteto De Nos,Ya No Sé Que Hacer Conmigo,0.502
96,J,1061051091011101011221221171081011169710610411...,El Cuarteto De Nos,Mario Neta,0.418
97,J,1061051091011101011221221171081011169710610411...,El Cuarteto De Nos,Punta Cana,0.948
98,J,1061051091011101011221221171081011169710610411...,Residente,Pecador,0.465
